In [ ]:
pip install tableschema

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.9/68.9 KB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 KB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.1/72.1 KB 6.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.7/132.7 KB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.8/114.8 KB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 59.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.6/79.6 KB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 11.4 MB/s eta 0:00:00
  Created wheel for unicodecsv: filename=unicodecsv-0.14.1-py3-none-any.whl size=10766 sha256=ead787f36263166e83e7d57baf94653437710714969d34bf98504de4370ea563
  Stored in dire

In [ ]:
import re
import json
from google.colab import files
import pandas as pd 
df= pd.read_csv("HAD.csv",sep=",",encoding='utf-8',na_values="")
df.rename(columns={'Nom variable':'variables','Observation CNAMTS':'observation','Libellé':'description','Longueur':'length', 'Année de création':'dateCreated', 'Année avant disparition':'dateDeleted',  'Description table':'tab_lib'}, inplace=True)
df=df.fillna('')



##### transformation des types 
mapping_t = {
    "Char": "string",
    "char":"string",
    "Num":"integer",
    "num":"integer",
    "date":"date",
    "Date":"date"
  }
df["type"]=df["Type"].map(mapping_t)
  # Fonction qui permet de classer le type en number ou integer en fct de la taille 
def numbint(x): 
  res = re.search(",", x)
  if res:
    return "number"
  return "integer"

def strdate(x):
  if 'MOI' in x:
    return "date"
  return "string"

df["datemissing"]=''

df['nomenclature']='-'
df["rdg"]=''

  


 # Creation dy type oracle 
mapping_to = {
    "string": "string",
    "date":"date",
    "integer":"number"
    
    }

df["type_oracle"]=df["type"].map(mapping_to)

 
# Creation des bons types
for i in range(len(df['variables'])):
  if df['type'][i] =='number':
    df['type'][i]=numbint(df['length'][i])  
  elif df['type'][i]=='string':
    df['type'][i]=strdate(df['variables'][i])

  

# create json file with the tables name
tableexist={
    'table':[],
    'tab_lib':[],
    'champ':[],
    'produit':'PMSI HAD'
}


for z in range(len(df['variables'])):
  if df['dateDeleted'][z] is not '':
    df['dateDeleted'][z]=df['dateDeleted'][z]+1

df['datereated']=''
for z in range(len(df['variables'])):
   # Transformation of type 
  if df['dateCreated'][z] is not '':
    df['dateCreated'][z] = int(float(df['dateCreated'][z]))
  if df['dateDeleted'][z] is not '':  
    df['dateDeleted'][z] = int (float(df['dateDeleted'][z]))
  df=df.fillna('')
  if df['Table'][z] in tableexist['table']: 
    
    new_dict={
              "name": df["variables"][z],
              "description": df["description"][z],
              "type": df["type"][z],
              "nomenclature": df["nomenclature"][z],
              "length": str(df["length"][z]),
              "format": "default",
              "dateCreated": str(df["dateCreated"][z]),
              "dateDeleted": str(df["dateDeleted"][z]),
              "dateMissing": [],
              "observation": df["observation"][z],
              "regle_gestion": df["rdg"][z],
              "type_oracle": df["type_oracle"][z]
        
          }
    
    with open('{}.json'.format(df['Table'][z]),encoding='utf-8') as f:  
      data=json.load(f)
      data.append(new_dict)
    with open('{}.json'.format(df['Table'][z]), 'w', encoding='utf-8') as f:
      json.dump(data, f, indent=4, ensure_ascii=False)
  else:
    with open('{}.json'.format(df['Table'][z]), 'w', encoding='utf-8') as f:
      json.dump([
            {"name": df["variables"][z],
              "description": df["description"][z],
              "type": df["type"][z],
              "nomenclature": df["nomenclature"][z],
              "length": str(df["length"][z]),
              "format": "default",
              "dateCreated": str(df["dateCreated"][z]),
              "dateDeleted": str(df["dateDeleted"][z]),
              "dateMissing": [],
              "observation": df["observation"][z],
              "regle_gestion": df["rdg"][z],
              "type_oracle": df["type_oracle"][z]}
        ], f, indent=4, ensure_ascii=False)
      tableexist['table'].append(df['Table'][z])
      tableexist['tab_lib'].append(df['tab_lib'][z])
     
  # Mettons les fichiers sous format tableschema

from tableschema import Schema
for table in tableexist['table']:
  with open('{}.json'.format(table), "r", encoding='utf-8') as f:
    descriptor = {'fields': json.load(f)}
    schema=Schema(descriptor) 
    schema.descriptor['name']=table
    schema.descriptor['title']=tableexist['tab_lib'][tableexist['table'].index(table)]
    schema.descriptor['produit']=tableexist['produit']
    schema.descriptor['missingValues'] = ['']
    if table=='T_HADaaB':
      schema.descriptor['primaryKey']= [
        "ETA_NUM_ePMSI",
        "RHAD_NUM"
        ]
      schema.descriptor['foreignKeys'] = [{"fields":["ETA_NUM"],
                                           "reference": {
                                             "resource": "T_HADaaE",
                                             "fields":["ETA_NUM"]
                                             }
                                          }
                                        
                                         ]
    elif table=='T_HADaaE':
      schema.descriptor['primaryKey']=['ETA_NUM']
    elif table=='T_HADaaC':
      schema.descriptor['primaryKey']= [
        "ETA_NUM_EPMSI",
        "RHAD_NUM"
        ]
      schema.descriptor['foreignKeys']=[{"fields":[
                                          "ETA_NUM_EPMSI",
                                          "RHAD_NUM"
                                       ],
                                       "reference": {
                                             "resource": "T_HADaaB",
                                             "fields": [
                                                "ETA_NUM_EPMSI",
                                                "RHAD_NUM"
                                            ]
                                            },
                                       "description" : "Bidirectionnel"
                                       },
                                      {
                                          "fields": "NIR_ANO_17", 
                                          "reference": {
                                             "resource": "IR_BEN_R",
                                             "fields": "BEN_NIR_PSA"
                                      }
                                      }
                                      ]
    elif table=='T_HADaaEHPA':
      schema.descriptor['foreignKeys']=[{"fields":[
                                          "ETA_NUM_EPMSI",
                                          "RHAD_NUM_EHPA"
                                       ],
                                       "reference": {
                                             "resource": "T_HADaaE",
                                             "fields": [
                                                "ETA_NUM",
                                                "ETA_NUM"
                                            ]
                                            }
                                       }
                                      ]
    else:
      schema.descriptor['foreignKeys']=[{"fields":[
                                          "ETA_NUM_EPMSI",
                                          "RHAD_NUM"
                                       ],
                                       "reference": {
                                             "resource": "T_HADaaB",
                                             "fields": [
                                                "ETA_NUM_EPMSI",
                                                "RHAD_NUM"
                                            ]
                                            }
                                       }
                                      ]
    schema.descriptor['history']={"dateCreated": schema.descriptor['fields'][0]['dateCreated'],
                                    "dateDeleted": "",
                                    "dateMissing": []}
    schema.descriptor['champ']=" Public et Privé"
    schema.descriptor['observation']=""                               
    schema.commit()
    schema.save('{}.json'.format(table),ensure_ascii=False)
from google.colab import files

<>:71: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:77: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:79: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:71: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:77: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:79: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<ipython-input-4-f5b79b54a509>:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['type'][i]=strdate(df['variables'][i])
<ipython-input-4-f5b79b54a509>:71: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if df['dateDeleted'][z] is not '':
<ipython-input-4-f5b79b54a509>:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation

In [ ]:
from google.colab import files
!zip -r /content/sample_data.zip /content


  adding: content/ (stored 0%)
  adding: content/.config/ (stored 0%)
  adding: content/.config/.last_survey_prompt.yaml (stored 0%)
  adding: content/.config/.last_update_check.json (deflated 22%)
  adding: content/.config/logs/ (stored 0%)
  adding: content/.config/logs/2023.01.04/ (stored 0%)
  adding: content/.config/logs/2023.01.04/14.35.33.516796.log (deflated 55%)
  adding: content/.config/logs/2023.01.04/14.35.04.532501.log (deflated 54%)
  adding: content/.config/logs/2023.01.04/14.34.03.867433.log (deflated 92%)
  adding: content/.config/logs/2023.01.04/14.35.34.388081.log (deflated 54%)
  adding: content/.config/logs/2023.01.04/14.34.56.463059.log (deflated 86%)
  adding: content/.config/logs/2023.01.04/14.34.28.750487.log (deflated 53%)
  adding: content/.config/gce (stored 0%)
  adding: content/.config/.last_opt_in_prompt.yaml (stored 0%)
  adding: content/.config/active_config (stored 0%)
  adding: content/.config/configurations/ (stored 0%)
  adding: content/.config/conf

In [ ]:
files.download('/content/sample_data.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>